I use this notebook to automatically label image overlap pairs that I haven't seen yet with pairs that I have seen (See [tile_cliques_poc.ipynb](notebooks/eda/tile_cliques_poc.ipynb), and [tile_dicts_vs_cliques.ipynb](notebooks/eda/tile_dicts_vs_cliques.ipynb)).

In [ ]:
from collections import Counter

import numpy as np
import networkx as nx
from tqdm import tqdm_notebook

from sdcdup.utils import generate_tag_pair_lookup
from sdcdup.utils import load_duplicate_truth
from sdcdup.utils import update_duplicate_truth
from sdcdup.utils import update_tile_cliques

from sdcdup.features import SDCImageContainer
from sdcdup.features import load_image_overlap_properties

%reload_ext autoreload
%autoreload 2

img_overlap_index_maps = generate_tag_pair_lookup()

In [ ]:
sdcic = SDCImageContainer()
sdcic.preprocess_image_properties()

In [ ]:
dup_truth = load_duplicate_truth()
print(len(dup_truth))

In [ ]:
n_matching_tiles_list = [9, 6, 4, 3, 2, 1]
overlap_image_maps = load_image_overlap_properties(n_matching_tiles_list, sdcic)
print(len(overlap_image_maps))

## Get all the overlap_image_maps that are not in dup_truth.

In [ ]:
overlap_candidates = []
for (img1_id, img2_id), overlap_maps in tqdm_notebook(overlap_image_maps.items()):
    for img1_overlap_tag in overlap_maps:
        if (img1_id, img2_id, img1_overlap_tag) in dup_truth:
            continue
        overlap_candidates.append((img1_id, img2_id, img1_overlap_tag))
print(len(overlap_candidates))

## Create list of flat hashes. 
(i.e. hashes for tiles where every pixel is the same color)

In [ ]:
solid_hashes = set()
for img_id, tile_issolid_grid in sdcic.tile_issolid_grids.items():
    idxs = set(np.where(tile_issolid_grid >= 0)[0])
    for idx in idxs:
        if np.all(tile_issolid_grid[idx] >= 0):
            solid_hashes.add(sdcic.tile_md5hash_grids[img_id][idx])

print(solid_hashes)

### Using cliques (networkx)

In [ ]:
tile_hash_dup_cliques = nx.Graph()
tile_hash_dif_cliques = nx.Graph()

for (img1_id, img2_id, img1_overlap_tag), is_dup in dup_truth.items():
    for idx1, idx2 in img_overlap_index_maps[img1_overlap_tag]:
        tile1_hash = sdcic.tile_md5hash_grids[img1_id][idx1]
        tile2_hash = sdcic.tile_md5hash_grids[img2_id][idx2]
        if is_dup:
            if tile1_hash in solid_hashes or tile2_hash in solid_hashes:
                continue
            update_tile_cliques(tile_hash_dup_cliques, tile1_hash, tile2_hash)
        else:
            if tile1_hash == tile2_hash:
                continue
            tile_hash_dif_cliques.add_edge(tile1_hash, tile2_hash)

print(tile_hash_dup_cliques.number_of_nodes(), tile_hash_dif_cliques.number_of_nodes())

neighbor_counts = Counter()
for tile_hashes in nx.connected_components(tile_hash_dup_cliques):
    neighbor_counts[len(tile_hashes)] += 1
list(sorted(neighbor_counts.items()))

In [ ]:
auto_overlap_labels = {}

for img1_id, img2_id, img1_overlap_tag in overlap_candidates:
    if (img1_id, img2_id, img1_overlap_tag) in auto_overlap_labels:
        continue
    is_dup = 1
    for idx1, idx2 in img_overlap_index_maps[img1_overlap_tag]:
        
        tile1_hash = sdcic.tile_md5hash_grids[img1_id][idx1]
        tile2_hash = sdcic.tile_md5hash_grids[img2_id][idx2]
        
        if tile1_hash in tile_hash_dif_cliques and tile2_hash in set(nx.neighbors(tile_hash_dif_cliques, tile1_hash)):
            is_dup = 0
            break
        elif tile1_hash in tile_hash_dup_cliques and tile2_hash in set(nx.neighbors(tile_hash_dup_cliques, tile1_hash)):
            continue
        else:
            is_dup = -1

    if is_dup == -1:
        continue
    
    auto_overlap_labels[(img1_id, img2_id, img1_overlap_tag)] = is_dup

print(len(auto_overlap_labels))

This will create a new txt file with the prefix `chunk_auto`, and then the date, followed by `len(auto_overlap_labels)`.  The new file will be saved to [data/processed/](data/processed/). TODO: example.

In [ ]:
dup_truth = update_duplicate_truth(auto_overlap_labels)
len(dup_truth)